In [3]:
import pandas as pd
from collections import Counter
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import json

In [2]:
# Ensure the necessary NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/c.ryan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/c.ryan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
file_path = "data/prizes-1000.json"

In [6]:
with open(file_path, 'r') as file:
    data = json.load(file)

In [11]:
data['nobelPrizes'][:1]

[{'awardYear': '1901',
  'category': {'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'},
  'categoryFullName': {'en': 'The Nobel Prize in Chemistry',
   'no': 'Nobelprisen i kjemi',
   'se': 'Nobelpriset i kemi'},
  'dateAwarded': '1901-11-12',
  'prizeAmount': 150782,
  'prizeAmountAdjusted': 10531894,
  'links': [{'rel': 'nobelPrize',
    'href': 'https://api.nobelprize.org/2/nobelPrize/che/1901',
    'action': 'GET',
    'types': 'application/json'}],
  'laureates': [{'id': '160',
    'knownName': {'en': "Jacobus H. van 't Hoff"},
    'fullName': {'en': "Jacobus Henricus van 't Hoff"},
    'portion': '1',
    'sortOrder': '1',
    'motivation': {'en': 'in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions',
     'se': 'såsom ett erkännande av den utomordentliga förtjänst han inlagt genom upptäckten av lagarna för den kemiska dynamiken och för det osmotiska trycket i lösningar'},
    'links': [{'

In [9]:
prizes = pd.DataFrame(data['nobelPrizes'])

In [10]:
prizes.head()

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation
0,1901,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}","{'en': 'The Nobel Prize in Chemistry', 'no': '...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN
1,1901,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",1901-11-14,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN
4,1901,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",{'en': 'The Nobel Prize in Physiology or Medic...,1901-10-30,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN


In [51]:
nan_entries = prizes[prizes['laureates'].apply(lambda x: isinstance(x, float))]

In [53]:
nan_entries.head(1)

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation
66,1914,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",NaN,146900,8682167,"[{'rel': 'nobelPrize', 'href': 'https://api.no...",NaN,{'en': 'No Nobel Prize was awarded this year. ...


In [80]:
# Remove the entires with no laureates
prizes_no_nan = prizes.dropna(subset=['laureates'])

In [81]:
prizes_no_nan.head()

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation
0,1901,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}","{'en': 'The Nobel Prize in Chemistry', 'no': '...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN
1,1901,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",1901-11-14,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN
4,1901,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",{'en': 'The Nobel Prize in Physiology or Medic...,1901-10-30,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN


In [56]:
max_length = prizes_no_nan['laureates'].apply(len).max()
print(max_length)

3


In [70]:
prizes_no_nan['laureates'].iloc[0][0].get('motivation').get('en')

'in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions'

In [83]:
prizes_no_nan['category'].iloc[0]

{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}

In [84]:
prizes_no_nan.loc[:, 'simple_category'] = prizes_no_nan['category'].apply(lambda x: x.get('en'))

In [86]:
prizes_no_nan.head()

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation,simple_category
0,1901,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}","{'en': 'The Nobel Prize in Chemistry', 'no': '...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN,Chemistry
1,1901,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",1901-11-14,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN,Literature
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN,Peace
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN,Physics
4,1901,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",{'en': 'The Nobel Prize in Physiology or Medic...,1901-10-30,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN,Physiology or Medicine


In [98]:
prizes_exploded = prizes_no_nan.explode('laureates')

In [99]:
prizes_exploded.head()

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation,simple_category
0,1901,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}","{'en': 'The Nobel Prize in Chemistry', 'no': '...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '160', 'knownName': {'en': 'Jacobus H. ...",NaN,Chemistry
1,1901,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",1901-11-14,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '569', 'knownName': {'en': 'Sully Prudh...",NaN,Literature
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '462', 'knownName': {'en': 'Henry Dunan...",NaN,Peace
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '463', 'knownName': {'en': 'Frédéric Pa...",NaN,Peace
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '1', 'knownName': {'en': 'Wilhelm Conra...",NaN,Physics


In [107]:
prizes_exploded['laureates'].iloc[0].get('motivation').get('en')

'in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions'

In [108]:
prizes_exploded.loc[:, 'motivation'] = prizes_exploded['laureates'].apply(lambda x: x.get('motivation').get('en'))

In [109]:
prizes_exploded.head()

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation,simple_category,motivation
0,1901,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}","{'en': 'The Nobel Prize in Chemistry', 'no': '...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '160', 'knownName': {'en': 'Jacobus H. ...",NaN,Chemistry,in recognition of the extraordinary services h...
1,1901,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",1901-11-14,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '569', 'knownName': {'en': 'Sully Prudh...",NaN,Literature,in special recognition of his poetic compositi...
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '462', 'knownName': {'en': 'Henry Dunan...",NaN,Peace,for his humanitarian efforts to help wounded s...
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '463', 'knownName': {'en': 'Frédéric Pa...",NaN,Peace,for his lifelong work for international peace ...
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","{'id': '1', 'knownName': {'en': 'Wilhelm Conra...",NaN,Physics,in recognition of the extraordinary services h...


In [111]:
prizes_simple = prizes_exploded[['awardYear', 'simple_category', 'motivation']]

In [112]:
prizes_simple.head()

,awardYear,simple_category,motivation
0,1901,Chemistry,in recognition of the extraordinary services h...
1,1901,Literature,in special recognition of his poetic compositi...
2,1901,Peace,for his humanitarian efforts to help wounded s...
2,1901,Peace,for his lifelong work for international peace ...
3,1901,Physics,in recognition of the extraordinary services h...


In [113]:
prizes_simple.to_csv('data/prizes_motivation.csv', index=False)

___

In [87]:
physics_prizes = prizes_no_nan[prizes_no_nan['simple_category'] == 'Physics']

In [89]:
physics_prizes.head()

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation,simple_category
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN,Physics
8,1902,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1902-11-11,141847,9907798,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '2', 'knownName': {'en': 'Hendrik A. L...",NaN,Physics
13,1903,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1903-11-12,141358,9583241,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '4', 'knownName': {'en': 'Henri Becque...",NaN,Physics
18,1904,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1904-11-08,140859,9549412,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '8', 'knownName': {'en': 'Lord Rayleig...",NaN,Physics
23,1905,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",NaN,138089,9361622,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '9', 'knownName': {'en': 'Philipp Lena...",NaN,Physics


In [91]:
physics_prizes.iloc[0]['laureates']

[{'id': '1',
  'knownName': {'en': 'Wilhelm Conrad Röntgen'},
  'fullName': {'en': 'Wilhelm Conrad Röntgen'},
  'portion': '1',
  'sortOrder': '1',
  'motivation': {'en': 'in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him',
   'se': 'såsom ett erkännande av den utomordentliga förtjänst han inlagt genom upptäckten av de egendomliga strålar, som sedermera uppkallats efter honom'},
  'links': [{'rel': 'laureate',
    'href': 'https://api.nobelprize.org/2/laureate/1',
    'action': 'GET',
    'types': 'application/json'}]}]

In [92]:
physics_motivations = [laureate['motivation']['en'] for laureates in physics_prizes['laureates'] for laureate in laureates if isinstance(laureates, list)]

In [93]:
physics_motivations[:5]

['in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him',
 'in recognition of the extraordinary service they rendered by their researches into the influence of magnetism upon radiation phenomena',
 'in recognition of the extraordinary service they rendered by their researches into the influence of magnetism upon radiation phenomena',
 'in recognition of the extraordinary services he has rendered by his discovery of spontaneous radioactivity',
 'in recognition of the extraordinary services they have rendered by their joint researches on the radiation phenomena discovered by Professor Henri Becquerel']

In [94]:
print(len(physics_motivations))
print(len(physics_prizes))

225
117


In [95]:
with open('data/physics_motivations.txt', 'w') as file:
    for motivation in physics_motivations:
        file.write(motivation + '\n')